# Tratamento dos dados

Tratando e formatando os dados coletados no [notebook 1](1-coleta.ipynb).

In [98]:
from datetime import datetime
import locale
import re

import pandas as pd

In [2]:
locale.setlocale(locale.LC_TIME, 'pt_BR')

'pt_BR'

In [82]:
# o csv de dados brutos possui 16 colunas
cols = ['col%02d'%n for n in range(1, 17)]

raw = pd.read_csv('dados/raw-scrap.csv', names=cols)
raw.head()

,col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,col11,col12,col13,col14,col15,col16
0,campeonato-brasileiro-serie-a,2012,1,São Januário - Rio de Janeiro - RJ,Domingo,20 de Maio de 2012,18:30,2,Vasco da Gama - RJ,NaN,1.0,Grêmio - RS,NaN,NaN,0.0,0.0
1,campeonato-brasileiro-serie-a,2012,2,Pituaçu - Salvador - BA,Domingo,20 de Maio de 2012,18:30,0,Bahia - BA,NaN,0.0,Santos - SP,NaN,NaN,0.0,0.0
2,campeonato-brasileiro-serie-a,2012,3,Pacaembu - Sao Paulo - SP,Sábado,19 de Maio de 2012,18:30,1,Palmeiras - SP,NaN,1.0,Portuguesa - SP,NaN,NaN,0.0,0.0
3,campeonato-brasileiro-serie-a,2012,4,Orlando Scarpelli - Florianopolis - SC,Sábado,19 de Maio de 2012,21:00,2,Figueirense - SC,NaN,1.0,Náutico - PE,NaN,NaN,0.0,0.0
4,campeonato-brasileiro-serie-a,2012,5,Pacaembu - Sao Paulo - SP,Domingo,20 de Maio de 2012,16:00,0,Corinthians - SP,NaN,1.0,Fluminense - RJ,NaN,NaN,0.0,0.0


- col01: competição
- col02: temporada
- col03: id_partida
- col04: estádio [, cidade, uf]
- col05: dia da semana
- col06: data por extenso
- col07: hora
- col08: placar do tempo normal para a equipe mandante
- col09: nome da equipe mandante
- col10: jogadores marcaram gols a favor da equipe mandante
- col11: placar do tempo normal para a equipe visitante
- col12: nome da equipe visitante
- col13: jogadores que marcaram gols a favor da equipe visitante
- col14: jogadores que marcaram gol contra a própria equipe
- col15: placar de disputa de pênaltis para a equipe mandante
- col16: placar de disputa de pênaltis para a equipe visitante

In [83]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5214 entries, 0 to 5213
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   col01   5214 non-null   object 
 1   col02   5214 non-null   int64  
 2   col03   5214 non-null   int64  
 3   col04   5214 non-null   object 
 4   col05   5214 non-null   object 
 5   col06   5214 non-null   object 
 6   col07   5214 non-null   object 
 7   col08   5214 non-null   object 
 8   col09   5150 non-null   object 
 9   col10   3591 non-null   object 
 10  col11   5150 non-null   float64
 11  col12   5150 non-null   object 
 12  col13   2914 non-null   object 
 13  col14   312 non-null    object 
 14  col15   5149 non-null   float64
 15  col16   5148 non-null   float64
dtypes: float64(3), int64(2), object(11)
memory usage: 651.9+ KB


In [84]:
# lista as partidas que não ocorreram
# 01 do Brasileirão (queda do avião da Chapecoense)
# 63 da Copa do Brasil (pela regra da vitória do visitante que dispensa 2º jogo)
a = [float(x) for x in range(20)]
col08_numerico = raw['col08'].apply(lambda x: str(x).strip().replace('.', '').isdigit())
col11_numerico = raw['col11'].apply(lambda x: str(x).strip().replace('.', '').isdigit())
placar_numerico = col08_numerico * col11_numerico
no_jogo = raw[~placar_numerico]
no_jogo

,col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,col11,col12,col13,col14,col15,col16
1897,campeonato-brasileiro-serie-a,2016,378,Arena Condá - Chapeco - SC,Domingo,11 de Dezembro de 2016,17:00,Chapecoense - SC,NaN,Atlético - MG,NaN,NaN,0,0,NaN,NaN
3828,copa-brasil-masculino,2012,29,Presidente Vargas/CE - Fortaleza - CE,Quarta,14 de Março de 2012,20:30,Ceará - CE,NaN,Gama - DF,NaN,NaN,0,0,NaN,NaN
3830,copa-brasil-masculino,2012,31,João Lamego - Ipatinga - MG,Quarta,14 de Março de 2012,20:30,Betim - MG,NaN,Real Noroeste Capixaba - ES,NaN,NaN,0,0,NaN,NaN
3832,copa-brasil-masculino,2012,33,Alfredo Jaconi - Caxias do Sul - RS,Quarta,14 de Março de 2012,20:30,Juventude - RS,NaN,Operario Ferroviario Esporte C - PR,NaN,NaN,0,0,NaN,NaN
3839,copa-brasil-masculino,2012,40,Frasqueirão - Natal - RN,Quarta,14 de Março de 2012,20:30,A.b.c. - RN,NaN,Trem - AP,NaN,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4502,copa-brasil-masculino,2016,61,Nabi Abi Chedid - Braganca Paulista - SP,Terça,26 de Abril de 2016,19:15,Bragantino - SP,NaN,Brasília - DF,NaN,NaN,0,0,NaN,NaN
4509,copa-brasil-masculino,2016,68,Couto Pereira - Curitiba - PR,Quarta,27 de Abril de 2016,19:30,Coritiba - PR,NaN,Guarany - CE,NaN,NaN,0,0,NaN,NaN
4545,copa-brasil-masculino,2016,104,Mineirão - Belo Horizonte - MG,Terça,17 de Maio de 2016,21:30,Cruzeiro - MG,NaN,Londrina - PR,NaN,NaN,0,0,NaN,NaN
4547,copa-brasil-masculino,2016,106,Vila Belmiro - Santos - SP,Quarta,18 de Maio de 2016,19:30,Santos - SP,NaN,Galvez - AC,NaN,NaN,0,0,NaN,NaN


In [85]:
# dropando as partidas que não ocorreram
raw.drop(index=no_jogo.index, inplace=True)

In [201]:
partidas = pd.DataFrame()

In [202]:
partidas['competicao'] = raw['col01']
partidas['temporada'] = raw['col02']
partidas['id_partida'] = raw['col03']

In [203]:
local = raw['col04'].str.split(' - ', expand=True)
local.columns = ['estadio', 'cidade', 'uf']
partidas = pd.concat([partidas, local], axis=1)

In [204]:
dias_da_semana = ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado']
partidas['dia_semana'] = pd.Categorical(raw['col05'], categories=dias_da_semana, ordered=True)

In [205]:
partidas['data'] = raw['col06'].apply(lambda d: datetime.strptime(d, ' %d de %B de %Y').date())
partidas['hora'] = raw['col07'].apply(lambda h: datetime.strptime(h, '%H:%M').time())

In [206]:
def normaliza_nome(nome):
    nome = nome.strip().upper().replace('.', '')
    nome = re.sub('[EF]\s?C\s', '', nome)
    nome = nome.replace('FUTEBOL CLUBE', '').replace('ESPORTE CLUBE', '')
    com_acento = 'ÄÁÀÃÂËÉÈÊÏÍÌÎÖÓÒÕÔÜÚÙÛÇÑ'
    sem_acento = 'AAAAAEEEEIIIIOOOOOUUUUCN'
    table = nome.maketrans(com_acento, sem_acento)
    nome = nome.translate(table)
    nome = nome.replace('ATHLETICO', 'ATLETICO')
    nome = nome.replace('C R B', 'CRB')
    nome = nome.replace('DE NATAL - RN', '- RN')\
                .replace('PARANAENSE - PR', '- PR')\
                .replace('CEARENSE - CE', '- CE')\
                .replace('MINEIRO - MG', '- MG')\
                .replace('DO PIAUI - PI', '- PI')
    nome = re.sub('\s+', ' ', nome)
    return nome

partidas['mandante'] = raw['col09'].apply(normaliza_nome)
partidas['visitante'] = raw['col12'].apply(normaliza_nome)
equipes = pd.concat([partidas['mandante'], partidas['visitante']]).unique()
equipes.sort()
'  *  '.join(equipes)

'4 DE JULHO - PI  *  7 DE SETEMBRO - MS  *  ABC - RN  *  AFOGADOS - PE  *  AGUIA - PA  *  AGUIA NEGRA - MS  *  AIMORE - RS  *  ALECRIM - RN  *  ALTOS - PI  *  AMADENSE - SE  *  AMERICA - MG  *  AMERICA - RN  *  AMERICANO - RJ  *  ANAPOLINA - GO  *  ANAPOLIS - GO  *  APARECIDENSE - GO  *  AQUIDAUANENSE - MS  *  ARACRUZ - ES  *  ARAPONGAS - PR  *  ASA - AL  *  ATLETICO - AC  *  ATLETICO - BA  *  ATLETICO - CE  *  ATLETICO - ES  *  ATLETICO - GO  *  ATLETICO - MG  *  ATLETICO - PR  *  AUDAX - SP  *  AUTO ESPORTE - PB  *  AVAI - SC  *  AVENIDA - RS  *  BAHIA - BA  *  BAHIA DE FEIRA - BA  *  BANGU - RJ  *  BARBALHA - CE  *  BETIM - MG  *  BOA - MG  *  BOAVISTA - RJ  *  BOAVISTA SPORT CLUB (ANTIGO BARREIRA) - RJ  *  BOTAFOGO - PB  *  BOTAFOGO - RJ  *  BRAGANTINO - PA  *  BRAGANTINO - SP  *  BRASIL - RS  *  BRASILIA - DF  *  BRASILIENSE - DF  *  BRUSQUE - SC  *  CABOFRIENSE - RJ  *  CALDENSE - MG  *  CAMETA - PA  *  CAMPINENSE - PB  *  CAPIVARIANO - SP  *  CASCAVEL - PR  *  CASTANHAL - PA  * 

In [207]:
partidas['mand_placar'] = raw['col08'].apply(lambda g: int(g))
partidas['visit_placar'] = raw['col11'].apply(lambda g: int(g))

In [208]:
mand_zero_penaltis = raw['col15'].fillna(0).apply(lambda x: x==0)
visit_zero_penaltis = raw['col16'].fillna(0).apply(lambda x: x==0)
zero_penaltis = mand_zero_penaltis * visit_zero_penaltis
raw[~zero_penaltis]

partidas['disputa_penaltis'] = ~zero_penaltis
partidas['mand_penaltis'] = raw['col15'].fillna(0).apply(lambda p: int(p))
partidas['visit_penaltis'] = raw['col16'].fillna(0).apply(lambda p: int(p))

In [209]:
partidas

,competicao,temporada,id_partida,estadio,cidade,uf,dia_semana,data,hora,mandante,visitante,mand_placar,visit_placar,disputa_penaltis,mand_penaltis,visit_penaltis
0,campeonato-brasileiro-serie-a,2012,1,São Januário,Rio de Janeiro,RJ,Domingo,2012-05-20,18:30:00,VASCO DA GAMA - RJ,GREMIO - RS,2,1,False,0,0
1,campeonato-brasileiro-serie-a,2012,2,Pituaçu,Salvador,BA,Domingo,2012-05-20,18:30:00,BAHIA - BA,SANTOS - SP,0,0,False,0,0
2,campeonato-brasileiro-serie-a,2012,3,Pacaembu,Sao Paulo,SP,Sábado,2012-05-19,18:30:00,PALMEIRAS - SP,PORTUGUESA - SP,1,1,False,0,0
3,campeonato-brasileiro-serie-a,2012,4,Orlando Scarpelli,Florianopolis,SC,Sábado,2012-05-19,21:00:00,FIGUEIRENSE - SC,NAUTICO - PE,2,1,False,0,0
4,campeonato-brasileiro-serie-a,2012,5,Pacaembu,Sao Paulo,SP,Domingo,2012-05-20,16:00:00,CORINTHIANS - SP,FLUMINENSE - RJ,0,1,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5209,copa-brasil-masculino,2021,118,Mineirão,Belo Horizonte,MG,Quarta,2021-10-20,21:30:00,ATLETICO - MG,FORTALEZA - CE,4,0,False,0,0
5210,copa-brasil-masculino,2021,119,Maracanã,Rio de Janeiro,RJ,Quarta,2021-10-27,21:30:00,FLAMENGO - RJ,ATLETICO - PR,0,3,False,0,0
5211,copa-brasil-masculino,2021,120,Arena Castelão,Fortaleza,CE,Quarta,2021-10-27,21:30:00,FORTALEZA - CE,ATLETICO - MG,1,2,False,0,0
5212,copa-brasil-masculino,2021,121,Mineirão,Belo Horizonte,MG,Domingo,2021-12-12,17:30:00,ATLETICO - MG,ATLETICO - PR,4,0,False,0,0


In [210]:
partidas.to_csv('dados/partidas.csv', index=False, encoding='utf8')

In [211]:
gols = raw[['col10', 'col13', 'col14']].fillna('').copy()
gols = pd.concat([dados, gols], axis=1)
for col in gols.columns[-3:]:
    gols[col] = gols[col].apply(lambda x: [g.strip() for g in x.split('|')])

In [212]:
def format_marcacao_gol(jogador_minuto_tempo):
    tempo = jogador_minuto_tempo[-4:-1]
    jogador_minuto = jogador_minuto_tempo[:-7]
    jogador = re.sub(r'[\d]{1,2}(\+[\d]{1,2})?$', '', jogador_minuto).strip()
    minuto = jogador_minuto.replace(jogador, '').strip()
    if minuto.find('+') > 0:
        minuto = 45.5
    minuto = float(minuto)
    if tempo=='2ºT':
        minuto += 45
    return [jogador, minuto]

In [213]:
registro_gols = []

for idx, row in gols.iterrows():
    if row['col10']!=['']:
        for m in row['col10']:
            registro = [row['competicao'], row['temporada'], row['id_partida'],
                        row['estadio'], 'mandante', row['mandante']]
            lado = 'pro'
            if m in row['col14']:
                lado = 'contra'
            jogador, minuto = format_marcacao_gol(m)
            registro += [lado, jogador, minuto]
            registro_gols.append(registro)
    if row['col13']!=['']:
        for n in row['col13']:
            registro = [row['competicao'], row['temporada'], row['id_partida'],
                        row['estadio'], 'visitante', row['visitante']]
            lado = 'pro'
            if n in row['col14']:
                lado = 'contra'
            jogador, minuto = format_marcacao_gol(n)
            registro += [lado, jogador, minuto]
            registro_gols.append(registro)

In [214]:
cols = ['competicao', 'temporada', 'id_partida', 'estadio', 'campo', 'equipe', 'pro_contra', 'jogador', 'minuto']
gols = pd.DataFrame(registro_gols, columns=cols)
gols.sort_values(by=['competicao', 'temporada', 'id_partida', 'minuto'], inplace=True)

In [215]:
gols

,competicao,temporada,id_partida,estadio,campo,equipe,pro_contra,jogador,minuto
0,campeonato-brasileiro-serie-a,2012,104,Beira-Rio,mandante,INTERNACIONAL - RS,pro,Elton,20.0
4,campeonato-brasileiro-serie-a,2012,104,Beira-Rio,visitante,ATLETICO - GO,pro,Renie,27.0
1,campeonato-brasileiro-serie-a,2012,104,Beira-Rio,mandante,INTERNACIONAL - RS,pro,Dagoberto,46.0
2,campeonato-brasileiro-serie-a,2012,104,Beira-Rio,mandante,INTERNACIONAL - RS,pro,Jakson Avelino,58.0
3,campeonato-brasileiro-serie-a,2012,104,Beira-Rio,mandante,INTERNACIONAL - RS,pro,Fred,76.0
...,...,...,...,...,...,...,...,...,...
10960,copa-brasil-masculino,2021,121,Mineirão,mandante,ATLETICO - MG,pro,Vargas,55.0
10961,copa-brasil-masculino,2021,121,Mineirão,mandante,ATLETICO - MG,pro,Vargas,68.0
10963,copa-brasil-masculino,2021,122,Arena da Baixada,visitante,ATLETICO - MG,pro,Keno,25.0
10964,copa-brasil-masculino,2021,122,Arena da Baixada,visitante,ATLETICO - MG,pro,Hulk,75.0


In [216]:
gols.to_csv('dados/gols.csv', index=False, encoding='utf8')